In [1]:
import tensorflow as tf
import numpy as np
import collections

tf.reset_default_graph()
sess = tf.InteractiveSession()

# Import data

In [2]:
def get_data(data_path=None):
    """Load raw data from data directory "data_path".
    Reads text file, converts strings to integer ids
    Args:
    data_path: string path to the directory
    Returns:
    tuple (raw_data, vocabulary)
    """
  
    data = tf.gfile.GFile(data_path, "r").read().replace("\n", "<eos>").split()
    
    counter = collections.Counter(data)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

    words, _ = list(zip(*count_pairs))
    word_to_id = dict(zip(words, range(len(words))))
    id_to_word = dict(zip(range(len(words)), words))

    data_in_ids = [word_to_id[word] for word in data if word in word_to_id]
    return data, data_in_ids, word_to_id, id_to_word

In [3]:
# converting a one-hot array into a word
def vec_to_word(vec, id_to_word):
    index = np.argmax(vec, axis=0) # get the index of the most probable word
    word = id_to_word[index]
    return word

In [4]:
raw_data_words, raw_data_ids, word_to_id, id_to_word = get_data("train.txt")
n_words = len(word_to_id)
print(n_words)

745


# Training

### Random batch function

In [5]:
# build a random batch from data
def get_batch(data, batch_size, time_steps, input_size):
    batch = np.zeros([batch_size, time_steps+1, input_size])
    for row in range(batch_size):
        t0 = np.random.randint(0, len(data)-time_steps) # starting time
        batch[row, :, :] = np.eye(input_size)[data[t0:t0+time_steps+1]]
    return batch[:, :-1, :], batch[:, 1:, :]

### Parameters

In [6]:
# Input size
batch_size = 5
time_steps = 10
#epochs = 1 # not considering epoch now

# max number of iterations
iterations = 200

# learning rate
n_neurons = 5
learning_rate = 0.1

### Set up RNN

In [7]:
# Input / Output(target)
X = tf.placeholder(tf.float32, [None, time_steps, n_words])
Y = tf.placeholder(tf.float32, [None, time_steps, n_words])

In [8]:
# Define a gru cell
cell = tf.nn.rnn_cell.GRUCell(n_words)
#cell = tf.contrib.rnn.OutputProjectionWrapper(
#    tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu),
#    output_size=n_words)

# Get gru cell output
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

# Final output layer
dense = tf.layers.dense(inputs=outputs, units=n_words, activation=None)

# Class probabilities
probs = tf.nn.softmax(dense)

### Define loss and optimizer

In [9]:
# Loss & optimizer
#loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=probs))
loss = tf.reduce_mean(tf.square(outputs - Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

# Training
train = optimizer.minimize(loss)

# Initializer
init = tf.global_variables_initializer()

### Run training

In [10]:
# Run the model
with tf.Session() as sess:
    sess.run(init)
    
    for iteration in range(iterations):
        
        x_batch, y_batch = get_batch(
            data=raw_data_ids,
            batch_size=batch_size,
            time_steps=time_steps,
            input_size=n_words)
        
        sess.run(train, feed_dict={X: x_batch, Y: y_batch})
        
        if iteration % 10 == 0:
            loss_ = loss.eval(feed_dict={X: x_batch, Y: y_batch})
            print(iteration, loss_)

0 0.0045956825
10 0.03775016
20 0.042112395
30 0.022589331
40 0.018119223
50 0.013709886
60 0.009618984
70 0.010496552
80 0.007123581
90 0.01038665
100 0.00651928
110 0.0062417127
120 0.0053795455
130 0.0051027513
140 0.0033686475
150 0.0031024183
160 0.0044150236
170 0.0047595785
180 0.0031650236
190 0.0028660947


# Test